In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')


Mounted at /content/gdrive


In [ ]:
import pandas as pd
import numpy as np
import os

#//////////////////////////////  INGESTION EMAIL \\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\

# Imposta la cartella in cui sono presenti i file CSV
folder_path = '/content/gdrive/MyDrive/dataset_email_GMAIL/'

# Inizializza il dataframe in cui verranno inseriti i dati
df_email_csv = pd.DataFrame()

# Itera su tutti i file nella cartella
for i, filename in enumerate(os.listdir(folder_path)):

    # Ho utilizzato questo per processare più file in contemporanea su diversi notebook
    #if (i==2): break
    #if( i<460): continue
    #if(i==480): break # ricordati che parte da 0 a contare, quindi se scrivi i==50, leggerà fino al dataframe 49

    if filename.startswith('export_log_gmail_') and filename.endswith('.csv'):
        # metti il file CSV in un dataframe temporaneo
        temp_df = pd.read_csv(os.path.join(folder_path, filename))

        if df_email_csv.empty:
            df_email_csv = temp_df
            #  sanity check
            print('dataset parziale numero ',i," :", len(df_email_csv))
        else:
            df_email_csv = pd.concat([df_email_csv, temp_df.iloc[1:]], ignore_index=True)
            #  sanity check
            print('dataset parziale numero ',i," :", len(df_email_csv))

# droppa il vecchio index e ne crea uno "nuovo"
df_email_csv.reset_index(drop=True, inplace=True)


# Stampa il dataframe finale
print('dataset completo: ',len(df_email_csv))



<ipython-input-2-2704b61ca7ad>:22: DtypeWarning: Columns (3,4,6,7) have mixed types. Specify dtype option on import or set low_memory=False.
  temp_df = pd.read_csv(os.path.join(folder_path, filename))


dataset parziale numero  460  : 1064670


<ipython-input-2-2704b61ca7ad>:22: DtypeWarning: Columns (3,4,6,7) have mixed types. Specify dtype option on import or set low_memory=False.
  temp_df = pd.read_csv(os.path.join(folder_path, filename))


dataset parziale numero  461  : 2127155


<ipython-input-2-2704b61ca7ad>:22: DtypeWarning: Columns (3,4,6,7) have mixed types. Specify dtype option on import or set low_memory=False.
  temp_df = pd.read_csv(os.path.join(folder_path, filename))


dataset parziale numero  462  : 3192568


<ipython-input-2-2704b61ca7ad>:22: DtypeWarning: Columns (3,4,6,7) have mixed types. Specify dtype option on import or set low_memory=False.
  temp_df = pd.read_csv(os.path.join(folder_path, filename))


dataset parziale numero  463  : 4257151


OSError: ignored

In [ ]:
#%%
##////////////////////////////DATASET SENZA GROUP \\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\
# Vado a splittare il dataset in email inviate al singolo ("NOGROUP") ed email inviate ai gruppi ("GROUP")

msg_ID_to_drop = df_email_csv.loc[(df_email_csv['source_from_header_address'] == 'Group') | (df_email_csv['source_address'] == 'Group') | (df_email_csv['destination_address'] == 'Group' ) | (df_email_csv['source_service'].isin(['groups', 'mailing-list-server'])) | (df_email_csv['destination_service'].isin(['mailing-list-server'])), 'message_id'].unique()

mask = df_email_csv['message_id'].isin(msg_ID_to_drop)
df_email_NOGROUP = df_email_csv[~mask]

#print('dataset completo NO_group: ',len(df_email_NOGROUP))


In [ ]:

# Funzione per filtrare i record ed ottenere solo le "email delivered", cioé quei record che mi permettono di identificare uno scambio tra un mittente ed un destinatario e quinid eliminare le informazioni inerenti tutto il viaggio/percorso delle email
def filtraggio(df):

    # lato mittente
    valid_source_service = ['gmail-ui', 'smtp-inbound', 'smtp-msa', 'smtp-relay']
    valid_source_selector = [None, np.nan,  'send', 'gmail-for-work']

    """
    Ho aggiunto: [None, np.nan] nella maschera dei servizi del destinatario perché ci sono delle email che si possono salvare con gmail-ui+send lato mittente e tuttto null lato destinatario
    """
    #lato destinatario
    valid_destination_service = [None, np.nan,'gmail-ui', 'smtp-outbound', 'smtp-outbound-to-gmail']

    #non mi serve perché tanto con quei service per forza dovrò avere questi selector
    #valid_destination_selector = [None, 'gmail-delivery-server', 'google-apps-for-work']

    #action type che identificato la ricezione corretta dell'email
    valid_action_type = [2, 3, 68]

    # Maska to filter out rows with invalid event targets
    mask_source_service = df['source_service'].isin(valid_source_service)
    mask_source_selector = df['source_selector'].isin(valid_source_selector)
    mask_destination_service = df['destination_service'].isin(valid_destination_service)
    #mask_destination_selector = df['destination_selector'].isin(valid_destination_selector)
    mask_action_type = df['action_type'].isin(valid_action_type)

    # Applico la maschera
    df_filtered = df[mask_source_service & mask_source_selector & mask_destination_service & mask_action_type]

    return df_filtered


In [ ]:
#//// ottengo il dataframe filtrato e quello scartato:

df_email_NOGROUP_filtrato = filtraggio(df_email_NOGROUP)


In [ ]:
# ////////////////////////////////////////////// ANALISI MITTENTI MULTIPLI \\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\

pivot= pd.pivot_table(df_email_NOGROUP_filtrato, index=["message_id"], values=["source_from_header_address"],   aggfunc=[ pd.Series.nunique], dropna=False)


source_id_da_controllare= pivot.loc[pivot[('nunique', 'source_from_header_address')]>1]
source_id_da_controllare = source_id_da_controllare.index.get_level_values('message_id')

# li scarto
df_email_NOGROUP_filtrato = df_email_NOGROUP_filtrato[ ~df_email_NOGROUP_filtrato['message_id'].isin(source_id_da_controllare)]


In [ ]:
#%%

##////////////////////////////DATASET CON GROUP \\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\

msg_ID_to_drop = df_email_csv.loc[(df_email_csv['source_from_header_address'] == 'Group') | (df_email_csv['source_address'] == 'Group') | (df_email_csv['destination_address'] == 'Group' ) | (df_email_csv['source_service'].isin(['groups', 'mailing-list-server'])) | (df_email_csv['destination_service'].isin(['mailing-list-server'])), 'message_id'].unique()

mask = df_email_csv['message_id'].isin(msg_ID_to_drop)
df_email_GROUP = df_email_csv[mask]



In [ ]:


def filtraggio_GROUP(df):

    # Define the list of valid event targets
    valid_source_service = ['gmail-ui','mailing-list-server', 'smtp-inbound', 'smtp-msa', 'groups']
    valid_source_selector = [None, np.nan, 'send', 'groups-ui']


    valid_destination_service = ['gmail-ui', 'smtp-outbound', 'smtp-outbound-to-gmail', None, np.nan,]


    #valid_destination_selector = [None, 'gmail-delivery-server', np.nan]

    valid_action_type = [2, 3, 68]

    # Create a mask to filter out rows with invalid event targets
    mask_source_service = df['source_service'].isin(valid_source_service)
    mask_source_selector = df['source_selector'].isin(valid_source_selector)
    mask_destination_service = df['destination_service'].isin(valid_destination_service)
    #mask_destination_selector = df['destination_selector'].isin(valid_destination_selector)
    mask_action_type = df['action_type'].isin(valid_action_type)


    #/////////////////////////////////////////////////////////////////////////////
    #togli Group dal mittente e dal destinatario
    mask_no_group_in_source_or_dest = ( ~(df['source_from_header_address'].isin(['Group'])) & ~(df['destination_address'].isin(['Group'])))


    #togli source=dest perché vuol dire che fa parte della mailing list
    #/////////////////////////////////////////////////////////////////////////////


    # Filter out invalid rows
    df_filtered = df[mask_source_service & mask_source_selector & mask_destination_service & mask_action_type & mask_no_group_in_source_or_dest]


    return df_filtered


In [ ]:
df_email_GROUP_filtrato = filtraggio_GROUP(df_email_GROUP)


In [ ]:
#%%

# ////////////////////////////////////////////// ANALISI MITTENTI MULTIPLI GROUP \\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\

#io sono interessato che ci sia un unico mittente per email
# mittenti per email
pivot1= pd.pivot_table(df_email_GROUP_filtrato, index=["message_id"], values=["source_from_header_address"],   aggfunc=[ pd.Series.nunique], dropna=False)

source_id_da_controllare= pivot1.loc[pivot1[('nunique', 'source_from_header_address')]>1]
source_id_da_controllare = source_id_da_controllare.index.get_level_values('message_id')


# SOLUZIONE AI MITTENTI MULTIPLI: droppo le email che hanno più di un mittente
df_email_GROUP_filtrato = df_email_GROUP_filtrato[ ~df_email_GROUP_filtrato['message_id'].isin(source_id_da_controllare)]


In [ ]:

#/////////////////////////////////////////////// UNISCO I DUE DATASET \\\\\\\\\\\\\\\\\\\

#Vado ad unire nuovamente i due dataset
df_email_completo = pd.concat([df_email_GROUP_filtrato, df_email_NOGROUP_filtrato], ignore_index=True)


In [ ]:

#DROP DUPLICATES

primary_key_cols = ['message_id', 'source_from_header_address', 'destination_address']

# Drop duplicate rows based on the primary key columns
df_email_completo = df_email_completo.drop_duplicates(subset=primary_key_cols)




In [ ]:
#////////////////////////// E-MAIL CON SOURCE E DEST UGUALI = \\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\


# le email con mittente e destinatario uguale non ci interessano, quindi le scarto
df_email_completo = df_email_completo[ ~(df_email_completo['source_from_header_address'] == df_email_completo['destination_address']) ]


In [ ]:
# STATISTICHE ESTRAZIONI

print('righe totali del dataset importato: ', len(df_email_csv))
print('righe totali del dataset completo dopo il cleaning: ', len(df_email_completo))
print()

print('email (message_id univoci) totali prima del cleaning: ', df_email_csv["message_id"].nunique())




#print(df_email_csv["message_id"].unique())
#print(df_email_completo["message_id"].unique())



# Drop NaN values from df_email_csv["message_id"]
csv_unique_ids = df_email_csv["message_id"].dropna().unique()

# Drop NaN values from df_email_completo["message_id"]
completo_unique_ids = df_email_completo["message_id"].dropna().unique()

# Now csv_unique_ids and completo_unique_ids will contain arrays without NaN values.
# You can proceed with finding the unique message_id values that are present in csv_unique_ids but not in completo_unique_ids.
missing_ids = pd.Series(list(set(csv_unique_ids) - set(completo_unique_ids)))

# Now, missing_ids will contain the unique "message_id" values present in df_email_csv but not in df_email_completo.
#print("message_id scartati: ",missing_ids)

print("email (message_id univoci) scartate: ", len(set(missing_ids)) )
print()

print("percentuale di email scartate sul dataset completo: ", int( (len(set(missing_ids)) / df_email_csv["message_id"].nunique()) *100 ) , "%"   )

# se viene intorno al 75% va bene, perché delle email normali ne scarto 1 ogni 2, mentre di quelle group ne scarto 2 ogni 3

In [ ]:
csv_file_path = '/content/gdrive/MyDrive/processati_email/email_460-479.csv'  # Specify the file path where you want to save the CSV
df_email_completo.to_csv(csv_file_path, index=False)

print("DataFrame df_email_completo saved to CSV successfully!")